In [ ]:
# Библиотеки 

In [2]:
import gc
import os
import time
import pandas as pd
import numpy as np
import re
from pymystem3 import Mystem
from pymorphy2 import MorphAnalyzer
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Выгрузка файлов

In [3]:
text_data = pd.read_csv("text_data_proc.csv")

In [ ]:
# Сортировка по дате

In [4]:
text_sort = text_data.sort_values(by=['datetime'],ascending=False,ignore_index = True)

In [12]:
text_sort['publication'][6907]

'🇷🇺 Новость: Whoosh запустил аренду городских электровелосипедов в Санкт-Петербурге, в скором времени средства передвижения появятся и в Московской области, сообщает ТАСС\\n\\n""Whoosh запускает компактные городские электровелосипеды в Московской области и Санкт-Петербурге. Новые модели уже доступны в Санкт-Петербурге, городской парк достигнет 1,5 тыс. велосипедов Whoosh. Еще 2,5 тыс. велосипедов поедут в Московскую область"", - сообщили в пресс-службе.\\n_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _\\n\\nИтог: пока особо не углублялся в бизнес этой компании, но аренда самокатов и велосипедов - дело сезонное.\\n\\n#wush #whoosh'

In [ ]:
# Лемматизация

In [41]:
morph = MorphAnalyzer()
text_sort_lemmatized_descriptions = []
for i, description in tqdm(enumerate(text_sort['publication'].values), total=len(text_sort)):
    lemmatized_description = [
        morph.parse(token)[0].normal_form for token in 
        re.findall(r'[а-яА-Я]+', description)
    ]
    text_sort_lemmatized_descriptions.append(lemmatized_description)
text_sort['lemmatized_descriptions'] = text_sort_lemmatized_descriptions

In [42]:
train = text_sort.head(1000) # Трейновая выборка

In [43]:
pubs = train.copy()

In [44]:
test = text_sort.iloc[1001:len(text_sort['publication'])] # Тестовая выборка

In [45]:
%%time
tfidf = TfidfVectorizer()

tfidf.fit([' '.join(str(word).lower() for word in sentence) for sentence in train['lemmatized_descriptions']])
train_x = tfidf.transform(
    [' '.join(str(word).lower() for word in sentence) for sentence in train['lemmatized_descriptions']]
)

print(f"train_x: {train_x.shape[0]:,} x {train_x.shape[1]:,}")

train_x: 1,000 x 6,279
CPU times: user 250 ms, sys: 14.4 ms, total: 264 ms
Wall time: 290 ms


In [46]:
test_x = tfidf.transform(
    [' '.join(str(word).lower() for word in sentence) for sentence in test['lemmatized_descriptions']]
)

print(f"test_x: {test_x.shape[0]:,} x {test_x.shape[1]:,}")

test_x: 62,938 x 6,279


In [47]:
train_x = pd.DataFrame(train_x.toarray())

In [48]:
test_x = pd.DataFrame(test_x.toarray())

In [49]:
names = tfidf.get_feature_names()

In [51]:
train_x.columns = names

In [52]:
test_x.columns = names

In [ ]:
# Выгрузка данных для моделей

In [59]:
train_x.to_csv('train_x.csv')

In [60]:
test_x.to_csv('test_x.csv')

In [ ]:
# Просмотр публикаций для создания таргета

In [18]:
for i in range(len(pubs)):
    print('_________________')
    print('NEXT',i, pubs[i])

_________________
NEXT 0 💬 Наблюдение: кстати говоря, фьючерсный рынок (товарная биржа  Сша) ожидает от Фрс следующих действий 👇\n\n1) Повышений ставки больше не ожидается.\n\n2) ФРС планирует удерживать ставку на пике до мая 2024 года.\n\n3) Первое снижение произойдет в мае 2024 на 0.25%\n\n4) К концу 2024 ставка будет снижена на 1.0%\n\nВ целом, всё, что закладывает рынок фьючерсов в цену индекса доллара (Dxy) и трежерис Сша по итогу отражается на монетарной политики Фрс. \n\nПоэтому ждём завершения роста ключевой ставки с конца года, и затем вынужденного снижения в рамках развивающейся мировой рецессии.\n\n#фрс #сша #макроэкономика #дкп
_________________
NEXT 1 😗😙😚😋 Самое важное за неделю. #weekly\n\n1. ЦБ с высокой долей вероятности, повысит ставку \nна заседании 15 сентября https://t.me/AK47pfl/15860\n2. Необычная и скандальная презентация Сбера о взгляде на будущее до 2035 года https://t.me/AK47pfl/15865\n3. Средняя зарплата в РФ стала почти в два раза ниже, \nчем в Китае https:/

NEXT 344 💬 Обозрение: кстати, Джаннет Йеллен глава Минфина Сша сморозила лайфхак на рынках в ближайшие 3 - 4 года. \n\nНачну с того, что на днях Йеллен чиновники задали вопрос, мол, что будет, если Китай из-за военной интервенции в Тайвань после начала введения жёстких санкций со стороны Сша будет сбрасывать гос. облигации Штатов со своего баланса? Йеллен ответил косвенно, мол имеется вероятность ограничения долларовой ликвидности в мире.\n\nПеревожу на русский язык, Джаннет сказала, что, мол, если китайцы начнут войну, в такой ситуации Штаты введу лютые санкции на Кнр и товарооборот между Сша и Кнр резко обвалится, на факторе чего мир получит серьёзный пинок инфляции. По сути будут следующие меры со стороны Сша: в Штатах будут созданы риски для дефолта, все страны мира (после китайско - американского кризиса) начнут продавать все гос. займы Сша, затем будет дефицит долларов в мире и как следствие растущий спрос на эту валюту. В совокупности с данными факторами мы получаем фатальный за

NEXT 594 $IMOEX ждем открытия. \nСильное сопротивление 3080-3100. \nПоддержка 3050-3015. \n\nТМК ОБЪЯВИЛА О СТАРТЕ SPO, АКЦИОНЕРЫ РАЗМЕСТЯТ 1,5% ОТ ОБЩЕГО ОБЪЕМА АКЦИЙ \nПОСЛЕ ЗАВЕРШЕНИЯ SPO ТМК ПРОВЕДЕТ ДОПЭМИССИЮ АКЦИЙ ПО ЗАКРЫТОЙ ПОДПИСКЕ В ПОЛЬЗУ ОСНОВНОГО АКЦИОНЕРА ""ТМК СТИЛ ХОЛДИНГ"" ПО ЦЕНЕ SPO \nЦеновой диапазон предложения составит от 205 до 235 руб. за одну акцию. Сбор заявок состоится в периоде с 21 по 28 сентября 2023 года.\n$TRMK. \nНе думаю, что здесь все просто. Надо внимательно последить, мутновато.
_________________
NEXT 595 Главное к открытию четверга (21.09):\n#брифинг\n\n📌 Значение индекса ЖиС: 67 (жадность). Подробнее об индексе здесь.\n\n📍 Курсы валют: 96.6 ₽/$ (+0.5%), 13.2 ₽/¥ (+0.3%). \n📍 Минфин предлагает ввести экспортную пошлину с 1 октября по 31 декабря 2023 года. Пошлины не коснутся нефти, нефтепродуктов, газа, лесоматериалов, лома и зерна.\n📍 Минпромторг предложил металлургам подумать над снижением стоимости арматуры для государственных и региональных ст

NEXT 844 _
_________________
NEXT 845 Всем привет. #обзорнадень\n\nМировые фондовые рынки торгуются в пятницу утром в зеленой зоне, нефть Brent +0,79% и находится около $94,42.\n\n✅СОБЫТИЯ:  \n\n13:30  🇷🇺RUB  Решение по процентной ставке (сент)  прогноз: 13,00%  пред: 12,00%\n\n-пройдет заседание совета директоров Транснефти. В повестке вопрос рекомендации по дроблению акций\n\nВ мире🌏:\n\n🇷🇺🇺🇸США произвели масштабное расширение санкционного списка по РФ. Под ограничения, в частности, попали автопроизводители ""АвтоВАЗ"" и ""Соллерс"", госкомпания ""Автодор""; банк ""Ак Барс"" и небольшой московский Синко-банк; ряд нефтесервисных компаний РФ, включая ""Газстройпром"", ""Газпром недра""; компания ""Ростелеком-Солар"". В тоже время  США продлили исключение из санкций для морской транспортировки нефти ""Сахалина-2"" в Японию. \n\n🇨🇳Промышленное производство Китая в августе и рост розничных продаж превзошли ожидания ➡️Объём промышленного производства (г/г) (авг)  факт: 4,5% прогноз: 4,0%  

In [ ]:
# Заполнение таргета происходит вручную
# Имя файла - target.csv

In [ ]:
# Выгрузка индексов публикаций для компаний

In [ ]:
alrosa_index=[]
for i in range(len(text_sort["publication"])):
    if ("Алроса" in text_sort["publication"][i]) or ("ALRS" in text_sort["publication"][i]) or ("алроса" in text_sort["publication"][i]):
        alrosa_index.append(i)
pd.DataFrame(alrosa_index).to_csv('alrosa_index.csv')

In [ ]:
oak_index=[]
for i in range(len(text_sort["publication"])):
    if ("Оак" in text_sort["publication"][i]) or ("оак" in text_sort["publication"][i]) or ("ОАК" in text_sort["publication"][i]) or ("UNAC" in text_sort["publication"][i]) or ("Объединенная авиастроительная корпорация" in text_sort["publication"][i]):
        oak_index.append(i)
pd.DataFrame(oak_index).to_csv('oak_index.csv')

In [ ]:
yndx_index=[]
for i in range(len(text_sort["publication"])):
    if ("YNDX" in text_sort["publication"][i]) or ("Yandex" in text_sort["publication"][i]) or ("Яндекс" in text_sort["publication"][i]) or ("яндекс" in text_sort["publication"][i]) or ("yandex" in text_sort["publication"][i]):
        yndx_index.append(i)
pd.DataFrame(yndx_index).to_csv('yndx_index.csv')

In [ ]:
smlt_index=[]
for i in range(len(text_sort["publication"])):
    if ("SMLT" in text_sort["publication"][i]) or ("Самолет" in text_sort["publication"][i]) or ("Самолёт" in text_sort["publication"][i]) or ("самолет" in text_sort["publication"][i]) or ("самолёт" in text_sort["publication"][i]):
        smlt_index.append(i)
pd.DataFrame(smlt_index).to_csv('smlt_index.csv')

In [ ]:
lenz_index=[]
for i in range(len(text_sort["publication"])):
    if ("LNZL" in text_sort["publication"][i]) or ("Лензолото" in text_sort["publication"][i]) or ("ЛЕНЗОЛОТО" in text_sort["publication"][i]) or ("лензолото" in text_sort["publication"][i]):
        lenz_index.append(i)
pd.DataFrame(lenz_index).to_csv('lenz_index.csv')

In [ ]:
sber_index=[]
for i in range(len(text_sort["publication"])):
    if ("SBER" in text_sort["publication"][i]) or ("Сбер" in text_sort["publication"][i]) or ("СБЕР" in text_sort["publication"][i]) or ("сбер" in text_sort["publication"][i]):
        sber_index.append(i)
pd.DataFrame(sber_index).to_csv('sber_index.csv')

In [ ]:
ozon_index=[]
for i in range(len(text_sort["publication"])):
    if ("OZON" in text_sort["publication"][i]) or ("Ozon" in text_sort["publication"][i]) or ("ОЗОН" in text_sort["publication"][i]) or ("озон" in text_sort["publication"][i]) or ("ozon" in text_sort["publication"][i]):
        ozon_index.append(i)
pd.DataFrame(ozon_index).to_csv('ozon_index.csv')

In [ ]:
cian_index=[]
for i in range(len(text_sort["publication"])):
    if ("CIAN" in text_sort["publication"][i]) or ("Cian" in text_sort["publication"][i]) or ("ЦИАН" in text_sort["publication"][i]) or ("циан" in text_sort["publication"][i]) or ("cian" in text_sort["publication"][i]):
        cian_index.append(i)
pd.DataFrame(cian_index).to_csv('cian_index.csv')

In [ ]:
hhru_index=[]
for i in range(len(text_sort["publication"])):
    if ("HHRU" in text_sort["publication"][i]) or ("HeadHunter" in text_sort["publication"][i]) or ("Хедхантер" in text_sort["publication"][i]) or ("хедхантер" in text_sort["publication"][i]) or ("headhunter" in text_sort["publication"][i]):
        hhru_index.append(i)
pd.DataFrame(hhru_index).to_csv('hhru_index.csv')

In [ ]:
luk_index=[]
for i in range(len(text_sort["publication"])):
    if ("LKOH" in text_sort["publication"][i]) or ("Лукойл" in text_sort["publication"][i]) or ("лукойл" in text_sort["publication"][i]) or ("ЛУКОЙЛ" in text_sort["publication"][i]):
        luk_index.append(i)
pd.DataFrame(luk_index).to_csv('luk_index.csv')

In [ ]:
rosn_index=[]
for i in range(len(text_sort["publication"])):
    if ("ROSN" in text_sort["publication"][i]) or ("Роснефть" in text_sort["publication"][i]) or ("роснефть" in text_sort["publication"][i]) or ("РОСНЕФТЬ" in text_sort["publication"][i]):
        rosn_index.append(i)
pd.DataFrame(rosn_index).to_csv('rosn_index.csv')

In [ ]:
wush_index=[]
for i in range(len(text_sort["publication"])):
    if ("WUSH" in text_sort["publication"][i]) or ("Whoosh" in text_sort["publication"][i]) or ("whoosh" in text_sort["publication"][i]):
        wush_index.append(i)
pd.DataFrame(wush_index).to_csv('wush_index.csv')

In [ ]:
# Выгрузка данных о дате и времени публикаций 

In [ ]:
text_sort['datetime'].to_csv('time_sorted.csv')

In [ ]:
# Выгрузка данных о частоте манупилятивных публикаций каналов

In [6]:
text_sort['channel id'].to_csv('channel_id_sorted.csv')